In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import cv2
import numpy as np
from utils import *
from frame import *
import matplotlib.pyplot as plt
import g2o
import pangolin
import OpenGL.GL as gl



In [2]:
np.set_printoptions(suppress=True)

In [3]:
# fe=Extractor()
# frames=[]

In [4]:
# def project (pose,pts):


In [5]:
class Map(object):
    def __init__(self):
        self.frames=[]
        self.points=[]
        self.viewer_init()

    def viewer_init(self):
        pangolin.CreateWindowAndBind('Main', 640, 480)
        gl.glEnable(gl.GL_DEPTH_TEST)

        self.scam = pangolin.OpenGlRenderState(
            pangolin.ProjectionMatrix(640, 480, 420, 420, 320, 240, 0.2, 100),
            pangolin.ModelViewLookAt(-2, 2, -2, 0, 0, 0, pangolin.AxisDirection.AxisY))
        self.handler = pangolin.Handler3D(self.scam)

        # Create Interactive View in window
        self.dcam = pangolin.CreateDisplay()
        self.dcam.SetBounds(0.0, 1.0, 0.0, 1.0, -640.0/480.0)
        self.dcam.SetHandler(self.handler)
    
    def viewer_refresh(self):
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        gl.glClearColor(1.0, 1.0, 1.0, 1.0)
        self.dcam.Activate(self.scam)
        
        # Render OpenGL Cube
        pangolin.glDrawColouredCube()

        # Draw Point Cloud
        # points = np.random.random((100000, 3)) * 10
        # colors = np.zeros((len(points), 3))
        # colors[:, 1] = 1 -points[:, 0] / 10.
        # colors[:, 2] = 1 - points[:, 1] / 10.
        # colors[:, 0] = 1 - points[:, 2] / 10.

        gl.glPointSize(10)
        gl.glColor3f(1.0, 0.0, 0.0)
        # access numpy array directly(without copying data), array should be contiguous.
        ppts=np.array([d[:3,3] for d in self.state[0]])
        pangolin.DrawPoints(ppts)
        #pangolin.DrawPoints(points)  

        gl.glPointSize(10)
        gl.glColor3f(0.0, 1.0, 0.0)
        # # access numpy array directly(without copying data), array should be contiguous.
        spts=np.array(self.state[1])

        pangolin.DrawPoints(spts)
        pangolin.FinishFrame()




    def display(self):
        poses,pts=[],[]
        for f in self.frames:
            poses.append(f.pose)
        for p in self.points:
            pts.append(p.pt)
        self.state=poses,pts


        self.viewer_refresh()

In [6]:
mapp=Map()

In [7]:

class Point(object):
    # apoint is a 3D point in the world
    #each Point is observed in multiple frame
    def __init__(self,mapp,loc):
        self.pt=loc #state vector
        self.frames=[]
        self.idxs=[]

        self.id=len(mapp.points)
        mapp.points.append(self)

    def add_observation(self,frame,idx):
        self.frames.append(frame)
        self.idxs.append(idx)

In [8]:


def process_img(img,depth):

    frame=Frame(mapp,img,depth)
    # frames.append(frame)

    if frame.id==0:
        return
    
    f1=mapp.frames[-1]
    f2=mapp.frames[-2]
    idx1,idx2,pose=match(f1,f2)

    
    if idx1 is None:
        return
    if idx2 is None:
        return
    
    if pose is None:
        return
    
    f1.pose=np.dot(pose,f2.pose)

    # print(frames[-1].pts)
    pt4ds=add_ones(f1.pts[idx1])
    # print(pt4d.shape)
    pt4ds=np.dot(f2.pose,pt4ds.T).T
    # print(idx1)
    # print(pt4ds.shape)

    for i,p in enumerate(pt4ds):
        pt=Point(mapp,p)
        pt.add_observation(f1,idx1[i])
        pt.add_observation(f2,idx2[i])
    
    # print(pt4d[:,0])
    
    

    # print(frames[-1].pose)

    for kp1,kp2 in zip(f1.pts[idx1],f2.pts[idx2]):
        u1,v1,_=map(lambda x: int(round(x)),kp1)
        u2,v2,_=map(lambda x: int(round(x)),kp2)

        cv2.circle(img,(u1,v1),color=(0,255,0),radius=3)
        cv2.line(img,(u1,v1),(u2,v2),color=(255,0,0))
    disp(img,"RGB")

    mapp.display()




In [9]:
if __name__ == "__main__":
    
    depth_paths='../rgbd_dataset_freiburg1_xyz/depth.txt'
    dlist=data(depth_paths)

    rgb_paths='../rgbd_dataset_freiburg1_xyz/rgb.txt'
    ilist=data(rgb_paths)

    dataset_path='../rgbd_dataset_freiburg1_xyz/'
    

    for i in range(len(dlist)):

        frame=cv2.imread(dataset_path+ilist[i])
        depth=cv2.imread(dataset_path+dlist[i],0)
        # print(frame.shape,depth.shape)


        process_img(frame,depth)
        # print(frame)

        if frame is None:
            print("End of frame")
            break

        
        disp(depth,"Depth")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


        
